(numpy)=
# Numpy

`Numpy` -- это широко используемая библиотека для вычислений с многомерными массивами. API большей частью вдохновлен `MATLAB` (великая и ужасная среда, язык и IDE для матричных вычислений), а теперь сам является примером для подражания API различных вычислительных пакетов.
Более последовательный гайд стоит посмотреть на [сайте библиотеки](https://numpy.org/devdocs/user).


## Массивы

In [ ]:
import numpy as np

a = np.array([1, 2, 3]) # создадим вектор
print(f"{a = }")

b = np.zeros((2, 2))
print(f"{b = }")

c = np.eye(3)
print(f"{c = }")

q = np.random.random((1, 20))
print(f"{q = }")

## Арифметические операции
Для удобства использования `np.ndarray` арифметические операторы определены так, чтобы соответствовать ожиданиям:

In [ ]:
a = np.array([1, 2, 3])
b = np.array([-1, 3, 4])

diff = a - b
print(f"{diff = }")

mult = a * b
print(f"{mult = }")

scalar_mult = a @ b
print(f"{scalar_mult = }")

## Indexing, slicing and sugar

`Numpy` поддерживает, кажется, все разумные варианты индексации:

In [ ]:
a = np.arange(16).reshape(4, 4)
print(f"{a = }")

# просто по индексам
print(f"\n{a[0, 1] = }")
print(f"{a[0][1] = }")

# по слайсам
print(f"\n{a[0, 1:3] = }")
print(f"{a[2] = }")
print(f"{a[2, :] = }")
print(f"{a[2, ...] = }")

# по маске
mask = (a % 3 == 0)
print(f"\n{mask = }")
print(f"{a[mask] = }")

first_rows = np.array([True, True, False, False])
print(f"\n{a[first_rows] = }")

Для работы с размерностями часто используются еще три конструкции: `None`, `...` (ellipsis, многоточие) и `:` (двоеточие).

In [ ]:
a = np.arange(16).reshape(4, 4)
print(f"{a = }")

# None добавляет ось размерности 1
print(f"\n{a[None].shape = }")
print(f"{a[:, :, None].shape = }")

# : превращается в slice (None), берет все элементы вдоль размерности
print(f"\n{a[2, :] = }")
print(f"{a[2, 0:None] = }")

# ... ellipsis, превращается в необходимое число двоеточий :,:,:
print(f"\n{a[...] = }")

# также ... удобен когда мы не знаем настоящий шейп массива или нужно не трогать несколько подряд идущих размерностей
z = np.arange(27).reshape(3, 3, 3)
print(f"\n{z[0, ..., 1] = }")
print(f"{z[0, :, 1] = }")

В целом в `NumPy` очень здорово реализованы методы `__getitem__`/`__setitem__`.

In [ ]:
a = np.array([1, 2, 3])
element = a[2]
print(f"{element = }")

a[2] = 5
print(f"{a = }")

Кроме того, мы можем делать индексацию по заданному условию с помощью `np.where`.

In [ ]:
# создадим вектор
a = np.array([2, 4, 6, 8])

selection = np.where(a < 5)
print(f"{selection = }")

# дополнительно можем передать два значения или вектора, при выполнении условия выбираются элементы из первого значения/вектора, при невыполнении -- из второго
a2 = np.where(a < 5, 2, a * 2)
print(f"{a2 = }")

# np.where работает и с многомерными массивами
b = np.array([[8, 8, 2, 6], [0, 5, 3, 4]])
b_mult = np.where(b < 4, b, 1)
print(f"{b_mult = }")

## Broadcasting

Что происходит, если мы хотим производить арифметические операции с массивами разных размеров?

In [ ]:
a = np.array([1, 2, 3])
k = 2
broad = a * k
print(f"{broad = }")

С точки зрения математики, ничего интересного тут не происходит: мы подразумевали умножение всего вектора на скаляр. Однако матричные операции в numpy справляются и с менее очевидными случаями, например, при сложении или вычитании вектора и скаляра:

In [ ]:
a = np.array([1, 2, 3])
k = 2
broad = a - k
print(broad)

В numpy приняты следующие правила работы с массивами разного размера:
1. размерности сравниваются справа налево;
2. два массива совместимы в размерности, если она одинаковая, либо у одного из массивов единичная;
3. вдоль отсутствующих размерностей происходит расширение повторением (`np.repeat`).

```{figure} /_static/linalgblock/numpy_work_with_arrays.png
:name: numpy_work_with_arrays
```

```{attention} Be aware
Автоматический броадкастинг легко приводит к ошибкам, так что лучше делать его самостоятельно в явной форме.
```


## Операции с плавающей точкой

Отдельно стоит поговорить про числа с плавающей точкой.
Число с плавающей точкой (или число с плавающей запятой) -- экспоненциальная форма представления вещественных (действительных) чисел, в которой число хранится в виде мантиссы и порядка (показателя степени). При этом число с плавающей точкой имеет фиксированную относительную точность и изменяющуюся абсолютную.
В результате одно и то же значение может выглядеть по-разному, если хранить его с разной точностью.

In [ ]:
f16 = np.float16("0.1")
f32 = np.float32(f16)
f64 = np.float64(f32)
print(f"{f16 = }, {f32 = }, {f64 = }")
print(f"{f16 == f32 == f64 = }")

f16 = np.float16("0.1")
f32 = np.float32("0.1")
f64 = np.float64("0.1")
print(f"{f16 = }, {f32 = }, {f64 = }")
print(f"{f16 == f32 == f64 = }")

Из-за этого для сравнения массивов с типом float используют `np.allclose`.

In [ ]:
print(f"{np.allclose([1e10,1e-7], [1.00001e10,1e-8]) = }")
print(f"{np.allclose([1e10,1e-8], [1.00001e10,1e-9]) = }")

## NumPy и линейная алгебра

В `Numpy` много удобных функций, которые позволяют упростить код. Приведем несколько примеров:

In [ ]:
# матрица с единицами по диагонали и с нулями в остальных ячейках
print(f"{np.eye(2, dtype=int) = }")
# есть возможность указать индекс диагонали
print(f"{np.eye(3, k=-1, dtype=int) = }")

# в Numpy есть свой генератор случайных чисел и векторов
print(f"{np.random.beta(1, 2) = }")
print(f"{np.random.randint(1, 5, (2, 3)) = }")

# Numpy позволяет заменить значений основной диагонали матрицы.
# внимание, эта функция работает in-place
a = np.random.randint(1, 5, (3, 3))
print(f"{a = }")
np.fill_diagonal(a, 4)
print(f"{a = }")

# Можно сделать и наоборот -- получить вектор значений диагонали матрица
print(f"{np.diag(a) = }")

### Решение систем линейных уравнений

`Numpy` позволяет решить систему линейных уравнений.

In [ ]:
a = np.array([[7, 4], [9, 8]])
b = np.array([5, 3])
solution = np.linalg.solve(a, b)
print(solution)

### Обращение матриц

`Numpy` дает возможность выполнить операцию обращения матриц.

In [ ]:
a = np.array([[1., 2.], [3., 4.]])
inv = np.linalg.inv(a)
print(inv)

### Собственные вектора и числа

Вычисление собственных векторов и чисел.

In [ ]:
print(np.linalg.eig(np.diag((1, 2, 3))))

## Мы вкратце рассмотрели

- основы работы с `NumPy`;
- индексацию в массивах;
- broadcasting массивов `NumPy`;
- операции с плавающей точкой;
- `NumPy` и примитивы линейной алгебры.